In [2]:
import sys
from pathlib import Path

# Add project root to Python path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [3]:
import dspy
from src.modules_v2 import IntegratedText2SQLModule, ViewSelectorModule, EntityResolverModule
from src.data_loader import load_golden_dataset, load_snowflake_views, create_dspy_examples
from src.metrics import calculate_metrics, print_evaluation_report


In [4]:
lm = dspy.LM("azure/gpt-4o")
dspy.configure(lm=lm)
print("✅ DSPy configured")

✅ DSPy configured


In [10]:
golden_df = load_golden_dataset("../data/golden_dataset_v1.csv")
snowflake_views = load_snowflake_views("../data/snowflake_view.json")
examples = create_dspy_examples(golden_df)

In [11]:
examples[1]

{'question': 'What is MICs current exposure in the USA in PE asset class?',
 'conversation_history': '[]',
 'expected_views': 'INVESTMENT_KPI_VW',
 'selected_views_actual': nan}

In [12]:
entity_resolve_module = EntityResolverModule(config={})
view_selector_module = ViewSelectorModule(config = {}, candidate_views = snowflake_views)

In [14]:
entity_resolve_prediction = entity_resolve_module(
                question=examples[1]['question'] 
            )

In [15]:
entity_resolve_prediction

Prediction(
    reasoning='1. **Identify terms in query that match entity candidates:**\n   - The query mentions "PE" which is a common abbreviation for "Private Equity".\n   - The query also mentions "USA", which is a country, but there is no COUNTRY entity type provided in the entity candidates.\n\n2. **Apply disambiguation rules based on context keywords:**\n   - The query includes "asset class", which suggests that "PE" should be matched to the ASSET_CLASS entity type.\n\n3. **Match terms to EXACT values from entity_mapper.json:**\n   - "PE" is expanded to "Private Equity" based on the glossary and matches exactly with the ASSET_CLASS value "Private Equity".\n\n4. **Validate: no partial matches, no JSON artifacts, proper capitalization:**\n   - The matched entity "Private Equity" is an exact match from the ASSET_CLASS list and is properly capitalized.\n\n5. **No COUNTRY entity type is provided, so "USA" cannot be resolved to an entity.**',
    resolved_entities='{"ASSET_CLASS": "Pr

In [17]:
view_selector_prediction = view_selector_module(
                question=examples[1]['question'],
                resolved_entities=entity_resolve_prediction.resolved_entities,
                conversation_history=examples[1]['conversation_history']
            )

In [19]:
print(view_selector_prediction.selected_views)
print(examples[1]['expected_views'])

INVESTMENT_KPI_VW
INVESTMENT_KPI_VW
